<a href="https://colab.research.google.com/github/momchilgeorgiev/2023DL/blob/main/Project2024DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout, MaxPool2D, Conv2D, Flatten, Rescaling, Resizing
from tensorflow.keras.models import Sequential, Model

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [16]:
tf.config.list_physical_devices('GPU')

[]

In [6]:
!nvidia-smi

Sat Jan 13 23:46:43 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.33                 Driver Version: 546.33       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1080      WDDM  | 00000000:02:00.0  On |                  N/A |
|  0%   52C    P8              13W / 198W |   1506MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download paultimothymooney/chest-xray-pneumonia
! unzip /content/chest-xray-pneumonia.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
unzip:  cannot find or open /content/chest-xray-pneumonia.zip, /content/chest-xray-pneumonia.zip.zip or /content/chest-xray-pneumonia.zip.ZIP.


# Project Notebook :)

## Main ideas

* time: 10.jan.2024 - 5.feb.2024
* has to be simple
* maybe somethings about biology

### Objective

Create a robust model that detects pediatric pnuemonia. By robust, I mean a model that isn't effected by rotations (like in [situs inversus](https://en.wikipedia.org/wiki/Situs_inversus), mirrored organs; or when a doctor may flip the x-ray picture), different brightness levels and other.

I have downloaded a [chest x-ray ds](https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/data) that can be used to identify pneumonia patients.

First I will create a simple CNN that can detedect the raw data from the ds. Then I would like to build on top of that by merging already trained NN (like ResNet101 or another, more specialized for medical/x-ray image classification) to create a model that generalizes better.

**Objectives:**
* [x] Wacth 2 lectures on NN testing and how to work with images

**Main objectives**
1. Load the data and create a simple CNN
  1. [x] Load images
  2. [x] Apply normalization
  3. [x] Create a CNN and train it.
  4. [ ] Discuss it
2.






### Load data

First I want to load my data from the mentioned ds. I will be using `tf.keras.utils.image_dataset_from_directory` to do so. The sets are split into 80/20 for training and vaidation. Another set has been separated beforehand for testing which has 624 images.

In [7]:
TRAIN_PATH = "./train/"
TEST_PATH = "./test"
AUTOTUNE = tf.data.AUTOTUNE

In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(TRAIN_PATH, validation_split = 0.2, subset = 'training', seed = 42)
val_ds = tf.keras.utils.image_dataset_from_directory(TRAIN_PATH, validation_split = 0.2, subset = 'validation', seed = 42)
test_ds = tf.keras.utils.image_dataset_from_directory(TEST_PATH, seed = 42)


Found 5216 files belonging to 2 classes.
Using 4173 files for training.
Found 5216 files belonging to 2 classes.
Using 1043 files for validation.
Found 624 files belonging to 2 classes.


Here I set the number of classes we would use for the output and also using `.cache` to optimize loading.

In [9]:
CLASS_NUM = 2
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [10]:
tf.keras.backend.clear_session()

The model is constructed by 4 convolutional layers with valid padding. After every conv layer there is a pool layer to optimize training time.

Flatten layer is used to feed the data into the dense layers at the end.

We also rescale the pixels (`Rescaling(1./255)`) into range [0, 1] as a form of normalization.

Hidden layers are using `relu` for activation since the function's efficiency and at the output I have set 'sigmod' for classification of 2 classes. Sigmoid is used since it gives a range of [0, 1] which can be used as a *probability* metric.

In [11]:
model = Sequential([

    Rescaling(1./255),

    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),
    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),

    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),
    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),

    Flatten(),
    Dense(32, activation = 'relu'),
    Dense(16, activation = 'relu'),
    Dense(CLASS_NUM, activation = 'sigmoid'),
])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      9248      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 32)        9

In [12]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

Fit the model. Epochs are 3, because during testing, this number of epochs created sufficient results (val_accuracy >= 0.9)

In [13]:
model.fit(train_ds, validation_data = val_ds, epochs = 3)

Epoch 1/3
131/131 [==============================] - 112s 818ms/step - loss: 0.3794 - accuracy: 0.8212 - val_loss: 0.2490 - val_accuracy: 0.9012
Epoch 2/3
131/131 [==============================] - 92s 703ms/step - loss: 0.1399 - accuracy: 0.9461 - val_loss: 0.1121 - val_accuracy: 0.9645
Epoch 3/3
131/131 [==============================] - 94s 716ms/step - loss: 0.0920 - accuracy: 0.9708 - val_loss: 0.1071 - val_accuracy: 0.9693


In [13]:
model.evaluate(train_ds)

131/131 [==============================] - 21s 163ms/step - loss: 0.0718 - accuracy: 0.9736


[0.07176674157381058, 0.9736400842666626]

In [12]:
model.save('pediatric_pneumonia_model_1.keras')

#### Discussion

The simple model (229 938 params) has test and val accuracy of 0.96, which is in the range of BOE but that doesn't mean it generalizes better. IRL images can have different brightness and slight rotations which throw off the NN.

I would like now to train it with images with different brightness, rotations and horizontal positions. There are 2 options:

Option 1 is to add random brightness, flip and rotation layers in the model's architecture.

Option 2 is to upsample the data by simply having duplicates of images but with augmentations. This, I think, won't suffice since the model will be training on duplicates instead of single-cases.

For this reason I will add augmentation layers to the model.

### CNN with image augmentations

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
IMG_SIZE = 128

In [ ]:
model = Sequential([
    Resizing(IMG_SIZE)
    Rescaling(1./255),

    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),
    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),

    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),
    Conv2D(32, 3, padding = 'valid', activation = 'relu'),
    MaxPool2D(),

    Flatten(),
    Dense(32, activation = 'relu'),
    Dense(16, activation = 'relu'),
    Dense(CLASS_NUM, activation = 'sigmoid'),
])

[]